In [1]:
import pandas as pd
from eventclass import *
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline
sns.set(rc={'figure.figsize':(11.7,8.27)})

# Investigating Fencer Mobility to Div 1 from ROC events 2021-2022 season
With the elimination of Div II and Div III NAC events from the USA Fencing schedule, many adults age 20-40 without a current C rating are wondering how they can compete at NAC events again in the future.

Members of the leadership at USA Fencing have indicated that the Regional Open Circuit (ROC) events meet the competitive needs of fencers in this category. We are using results data from ROC events over the 2021-2022 season to determine if fencers in this age bracket who are currently ineligible for Division 1 have a path to Division 1 competition through ROC events.

This data is courtesy of Dan Berke at FencingTime. The file in this repository named "ROCResults.csv" from FencingTime, along with publicly availble membership data, is used below. We hope that providing this code allows for transparency in our analysis.

## Data ingest
Here is a list of the columns provided by FencingTime

In [2]:
rocs_raw = pd.read_csv('ROCResults.csv')
rocs_raw.columns

Index(['tournament', 'eventStartTime', 'city', 'state', 'level', 'gender',
       'weapon', 'eventClass', 'place', 'name', 'club', 'division',
       'memberNum', 'classification', 'earned', 'qualifiedFor'],
      dtype='object')

## Matching States and Regions
This data has tournaments listed by what state they are in. Let's make sure we know what region each state is a part of. Region assignments taken from this website: https://www.usafencing.org/regional-info
N.B. Alaska and Hawaii are not assigned regions, but D.C. is a part of Region 6, so our record should have 49 entries.

In [3]:
region_1_states = ['WA','OR','ID','MT','WY']
region_2_states = ['ND','SD','NE','KS','MN','IA','MO','WI','IL','MI','IN','OH','KY']
region_3_states = ['ME','NH','VT','MA','NY','RI','CT','NJ','PA']
region_4_states = ['CA','NV','AZ','NM','UT','CO']
region_5_states = ['TX','LA','OK','AR']
region_6_states = ['WV','DE','MD','VA','DC','TN','NC','SC','GA','AL','MS','FL']
region2state = {1:region_1_states,2:region_2_states,3:region_3_states,4:region_4_states,5:region_5_states,6:region_6_states}
state2region = {}
for region,region_states in region2state.items():
    for state in region_states:
        state2region[state] = region
print(f'We have accounted for {len(state2region.keys())} states')
rocs_raw['event_region'] = rocs_raw['state'].map(state2region)

We have accounted for 49 states


## Consolidating ratings
Now let's ignore whether a rating was earned in the first or second half of the season (i.e. we will consider A21 and A22 just as earning an A at some point during the season). Note that this column will have "nan" (i.e. "not a number") values for fencers that did not earn a rating at the event; these values are ignored.

Similarly, when determining whether a fencer already had a Division 1 eligible rating, we will ignore what year they earned their original classification before the tournament.

In [4]:
season_ratings = {x:x[0] for x in rocs_raw.classification.unique()}
rocs_raw['classification_season'] = rocs_raw['classification'].map(season_ratings)
season_earned_ratings = {x:x[0] for x in rocs_raw.earned.dropna().unique()}
rocs_raw['earned_season'] = rocs_raw['earned'].map(season_earned_ratings)

## Consolidating Events
Add an additional column for Gender+Weapon combos

In [5]:
rocs_raw['gw_event'] = rocs_raw['gender']+rocs_raw['weapon']

# Tournament Opportunities
First, let's see what classification level of events were possible last season. That is, what overall event rating was possible at each event, regardless of what the final classification was (i.e. we account for the ratings of those who attended the tournament and disregard any upsets that changed the actual final event classifications.) This uses functions from eventclass.py based on https://askfred.net/Info/eventClass.php

In [6]:
tournament_names = rocs_raw.tournament.unique()
tournaments = []
regions = []
levels = []
events = []
possible_classifications = []
for tournament in tournament_names:
    tournament_df = rocs_raw[rocs_raw.tournament==tournament]
    region = tournament_df.event_region.to_list()[0]
    tournament_levels = tournament_df.level.unique()
    for level in tournament_levels:
        level_df = tournament_df[tournament_df.level==level]
        level_events = level_df.gw_event.unique()
        for event in level_events:
            event_df = level_df[level_df.gw_event==event]
            possible_classification = get_event_rating(event_df.classification_season.to_list())
            tournaments.append(tournament)
            regions.append(region)
            levels.append(level)
            events.append(event)
            possible_classifications.append(possible_classification)
classification_df = pd.DataFrame.from_dict({'tournament':tournaments,'region':regions,'level':levels,'event':events,'possible_classification':possible_classifications})

### All possible event classifications at ROC tournaments

In [7]:
CLASS_ORDER = ['A4','A3','A2','A1','B3','B2','B1','C3','C2','C1','D1','E1','X']
EVENT_ORDER = ['MensEpee','MensFoil','MensSaber','WomensEpee','WomensFoil','WomensSaber']
LEVEL_ORDER = ['DivIA','DivII']
RATINGS_ORDER = ['A','B','C','D','E']

In [8]:
CATEGORY_ORDERS = {'possible_classification':CLASS_ORDER,
                    'event':EVENT_ORDER,
                    'level':LEVEL_ORDER}
fig_all_class = px.histogram(classification_df,
                    x='possible_classification',
                    color='event',
                    facet_col='level',
                    category_orders=CATEGORY_ORDERS,
                    title='Possible Event Classifications per ROC Type in 2021-2022 Season')
fig_all_class.update_layout(width=1000,height=500)
fig_all_class.show()
fig_all_class.write_image('plots/all_roc_classifications.png')

### Regional tournament classification availability per weapon/gender combination

In [9]:
CATEGORY_ORDERS = {'possible_classification':CLASS_ORDER,
                    'event':EVENT_ORDER,
                    'region':[1,2,3,4,5,6]}

df_divia = classification_df[classification_df.level=='DivIA']
divia_class = px.histogram(df_divia,
                    x='possible_classification',
                    color='region',
                    facet_col='event',
                    category_orders=CATEGORY_ORDERS,
                    title='2021-2022 Div IA ROC Event Possible Classifications per Region',
                    facet_col_wrap=3)
divia_class.update_layout(width=1000,height=1000)
divia_class.update_xaxes(showticklabels=True)
divia_class.show()
divia_class.write_image('plots/divia_event_classifications.png')

df_divii = classification_df[classification_df.level=='DivII']
divii_class = px.histogram(df_divii,
                    x='possible_classification',
                    color='region',
                    facet_col='event',
                    category_orders=CATEGORY_ORDERS,
                    title='2021-2022 Div II ROC Event Possible Classifications per Region',
                    facet_col_wrap=3)
divii_class.update_layout(width=1000,height=1000)
divii_class.update_xaxes(showticklabels=True)
divii_class.show()
divii_class.write_image('plots/divii_event_classifications.png')


### Now we can compute the maximum possible number of new ratings earned per region

In [10]:
MAX_EARNED = {'A4':{'A':8,'B':8,'C':8,'D':8,'E':16},
                'A3':{'A':4,'B':4,'C':8,'D':8,'E':8},
                'A2':{'A':1,'B':3,'C':4,'D':2,'E':2},
                'A1':{'A':1,'B':1,'C':2,'D':2,'E':2},
                'B3':{'A':0,'B':4,'C':4,'D':8,'E':16},
                'B2':{'A':0,'B':1,'C':3,'D':4,'E':4},
                'B1':{'A':0,'B':1,'C':3,'D':2,'E':2},
                'C3':{'A':0,'B':0,'C':4,'D':4,'E':8},
                'C2':{'A':0,'B':0,'C':1,'D':3,'E':4},
                'C1':{'A':0,'B':0,'C':1,'D':3,'E':4},
                'D1':{'A':0,'B':0,'C':0,'D':1,'E':3},
                'E1':{'A':0,'B':0,'C':0,'D':0,'E':1},
                'X':{'A':0,'B':0,'C':0,'D':0,'E':0}}

classification_df['ratings_dict'] = classification_df['possible_classification'].map(MAX_EARNED)
classification_df[['A','B','C','D','E']] = pd.json_normalize(classification_df.ratings_dict)
classification_df.drop(['ratings_dict'],axis=1,inplace=True)

In [11]:
CATEGORY_ORDERS = {'event':EVENT_ORDER,
                    'rating':['A','B','C','D','E'],
                    'region':[1,2,3,4,5,6]}

m = classification_df.drop(['tournament','possible_classification'],axis=1)
m = m.groupby(['region','level','event']).sum().reset_index()
melted_divia = m[m.level=='DivIA'].melt(id_vars=['region','event'],value_vars=['A','B','C','D','E'],value_name='rating_count',var_name='rating')
divia_possible = px.histogram(melted_divia,
                    x='rating',
                    y='rating_count',
                    color='region',
                    facet_col='event',
                    category_orders=CATEGORY_ORDERS,
                    title='Ratings possible to earn in 2021-2022 DivIA Events per Region',
                    facet_col_wrap=3)
divia_possible.update_layout(width=1000,height=1000)
divia_possible.update_xaxes(showticklabels=True)
divia_possible.show()
divia_possible.write_image('plots/divia_possible.png')

melted_divii = m[m.level=='DivII'].melt(id_vars=['region','event'],value_vars=['A','B','C','D','E'],value_name='rating_count',var_name='rating')
divii_possible = px.histogram(melted_divii,
                    x='rating',
                    y='rating_count',
                    color='region',
                    facet_col='event',
                    category_orders=CATEGORY_ORDERS,
                    title='Ratings possible to earn in 2021-2022 DivII Events per Region',
                    facet_col_wrap=3)
divii_possible.update_layout(width=1000,height=1000)
divii_possible.update_xaxes(showticklabels=True)
divii_possible.show()
divii_possible.write_image('plots/divii_possible.png')

# Now let's see how many ratings were actually earned

In [20]:
CATEGORY_ORDERS = {'earned_season':['A','B','C','D','E'],
                    'gw_event':EVENT_ORDER,
                    'event_region':[1,2,3,4,5,6]}

rocs_divia = rocs_raw[rocs_raw.level=='DivIA']
rocs_divia = rocs_divia[rocs_divia.earned_season.notna()]
divia_earned = px.histogram(rocs_divia,
                    x='earned_season',
                    color='event_region',
                    facet_col='gw_event',
                    category_orders=CATEGORY_ORDERS,
                    title='Ratings earned in 2021-2022 DivIA ROC Events per Region',
                    facet_col_wrap=3)
divia_earned.update_layout(width=1000,height=1000)
divia_earned.update_xaxes(showticklabels=True)
divia_earned.show()
divia_earned.write_image('plots/divia_earned.png')

rocs_divii = rocs_raw[rocs_raw.level=='DivII']
rocs_divii = rocs_divii[rocs_divii.earned_season.notna()]
divii_earned = px.histogram(rocs_divii,
                    x='earned_season',
                    color='event_region',
                    facet_col='gw_event',
                    category_orders=CATEGORY_ORDERS,
                    title='Ratings earned in 2021-2022 DivII ROC Events per Region',
                    facet_col_wrap=3)
divii_earned.update_layout(width=1000,height=1000)
divii_earned.update_xaxes(showticklabels=True)
divii_earned.show()
divii_earned.write_image('plots/divii_earned.png')

## Mobility to Division 1
Let's find out how many people who were previously ineligible for Division 1 events (i.e. D, E, and U fencers) earned a rating at an ROC event that made them Div 1 eligible (i.e. earned an A, B, or C)

N.B. We remove duplicates after downselecting in case a fencer made the graduation step twice before their new rating was updated

In [13]:
current = ['D','E','U']
new = ['A','B','C']
mobile = rocs_raw.loc[(rocs_raw.classification_season.isin(current)) & (rocs_raw.earned_season.isin(new))]
mobile.drop_duplicates(subset=['memberNum'],inplace=True)
print(f'Number of fencers who graduated to Div 1 from any ROC: {len(mobile.index)}')

Number of fencers who graduated to Div 1 from any ROC: 108


/var/folders/60/jf88x30d0vvdk51_8h0vdrrh0000gp/T/ipykernel_23561/3780815218.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
CATEGORY_ORDERS = {'level':LEVEL_ORDER,
                    'gw_event':EVENT_ORDER,
                    'event_region':[1,2,3,4,5,6]}
all_mobility = px.histogram(mobile,
                    x='gw_event',
                    color='event_region',
                    facet_col='level',
                    category_orders=CATEGORY_ORDERS,
                    title='Div I Mobility of All Fencers at ROC Events per Region for 2021-2022 Season',
                    facet_col_wrap=3)
all_mobility.update_layout(width=1000,height=1000)
all_mobility.update_xaxes(showticklabels=True)
all_mobility.show()
all_mobility.write_image('plots/mobility_all.png')

## Mobility of Non-Vet Adult Fencers (age 20-40) to Division 1 from ROC events
Now we cross-reference the Div 1 mobile fencers to the membership list to get their birth years

In [15]:
members = pd.read_csv('members.csv')
mobile_with_ages = pd.merge(mobile,members[['Birthdate','Member #']],left_on='memberNum',right_on='Member #')
mobile_adults = mobile_with_ages[mobile_with_ages['Birthdate'].between(1983,2003)]
print(f'Number of adult (20-40) fencers who graduated to Div 1 from any ROC: {len(mobile_adults.index)}')

Number of adult (20-40) fencers who graduated to Div 1 from any ROC: 26


In [16]:
CATEGORY_ORDERS = {'level':LEVEL_ORDER,
                    'gw_event':EVENT_ORDER,
                    'event_region':[1,2,3,4,5,6]}
adult_mobility = px.histogram(mobile_adults,
                    x='gw_event',
                    color='event_region',
                    facet_col='level',
                    category_orders=CATEGORY_ORDERS,
                    title='Div I Mobility of Non-Vet Adults (age 20-40) Fencers at ROC Events per Region for 2021-2022 Season',
                    facet_col_wrap=3)
adult_mobility.update_layout(width=1000,height=1000)
adult_mobility.update_xaxes(showticklabels=True)
adult_mobility.show()
adult_mobility.write_image('plots/mobility_adults.png')